In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

In [3]:
from mpl_toolkits.axes_grid1 import AxesGrid

In [4]:
import pickle as pk

In [5]:
%load_ext autoreload
%autoreload 2

# Custom tools

In [6]:
import sys

In [7]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.posargb as gb

In [9]:
date = "2019_02_22"
base_path = "/home/pleroy/DATA/DIADEM/lff/Bizona_2019_02_22/Raw_data"
prefix = "Volume_sol"
nbDir = 12
RxTx = ["21", "31", "32", "41", "42", "43"]

# Get heights

In [10]:
heights = {}
for k in range(1, nbDir+1):
    filename = base_path + f"/{prefix}_{k}/PoSAR.cfg"
    CFG = gb.readCfg( filename )
    heights[f"{k}"] = CFG["H_nom"]

In [11]:
heights

{'1': 3.5,
 '2': 3.42,
 '3': 3.34,
 '4': 3.26,
 '5': 3.18,
 '6': 3.1,
 '7': 3.02,
 '8': 2.94,
 '9': 2.86,
 '10': 2.78,
 '11': 2.7,
 '12': 2.62}

In [12]:
def plotImgMosaic( imgOrig, extent, origin='upper', T=0 ):
    fig = plt.figure()
    grid = AxesGrid(fig, 111,  # similar to subplot(111)
                    nrows_ncols=(3, 2),
                    axes_pad=0.0,
                    share_all=True,
                    label_mode="all",
                    cbar_location="top",
                    cbar_mode="single",
                   )
    
    img = dict(imgOrig)

    for im in img:
        img[im] = np.squeeze(img[im])
        if T:
            img[im] = img[im].T
    
    ij = "21"
    toPlot = 20 * np.log10( np.abs( img[ij] ) )
    im = grid[0].imshow(toPlot, cmap='jet', extent=extent, origin=origin)
    grid[0].set_ylabel(ij)

    ij = "41"
    toPlot = 20 * np.log10( np.abs( img[ij] ) )
    im = grid[1].imshow(toPlot, cmap='jet', extent=extent, origin=origin)
    grid[1].set_ylabel(ij)
    grid[1].yaxis.set_ticks_position('right')
    grid[1].yaxis.set_label_position('right')

    ij = "31"
    toPlot = 20 * np.log10( np.abs( img[ij] ) )
    im = grid[2].imshow(toPlot, cmap='jet', extent=extent, origin=origin)
    grid[2].set_ylabel(ij)

    ij = "42"
    toPlot = 20 * np.log10( np.abs( img[ij] ) )
    im = grid[3].imshow(toPlot, cmap='jet', extent=extent, origin=origin)
    grid[3].set_ylabel(ij)
    grid[3].yaxis.set_ticks_position('right')
    grid[3].yaxis.set_label_position('right')

    ij = "32"
    toPlot = 20 * np.log10( np.abs( img[ij] ) )
    im = grid[4].imshow(toPlot, cmap='jet', extent=extent, origin=origin)
    grid[4].set_ylabel(ij)

    ij = "43"
    toPlot = 20 * np.log10( np.abs( img[ij] ) )
    im = grid[5].imshow(toPlot, cmap='jet', extent=extent, origin=origin)
    grid[5].set_ylabel(ij)
    grid[5].yaxis.set_ticks_position('right')
    grid[5].yaxis.set_label_position('right')

    grid.cbar_axes[0].colorbar(im)
    
    return fig, grid

# zCut

In [ ]:
xMin = -1.5
xMax = 1.5
yMin = 0
yMax = 7
extent_yx = [yMin, yMax, xMin, xMax]

In [ ]:
hRef   = 3.500
zCut   = 0.000
with open(base_path + f"/{date}_hRef{hRef:.3f}_zCut{zCut:.3f}.dat", 'rb') as file:
    img = pk.load(file)

In [ ]:
fig, grid = plotImgMosaic(img[hRef], extent=extent_yx, origin='lower')
title = f"{date} ground range, hRef = {hRef}, zCut = {zCut}"
fig.suptitle(title)

# xCut

In [ ]:
yMin = 0
yMax = 7
zMin = -0.5
zMax = 1.5
extent_yz = [yMin, yMax, zMin, zMax]

In [ ]:
hRef   = 3.500
xCut   = 0.000
with open(base_path + f"/{date}_hRef{hRef:.3f}_xCut{xCut:.3f}.dat", 'rb') as file:
    img = pk.load(file)

In [ ]:
fig, grid = plotImgMosaic(img[hRef], extent=extent_yz, origin='lower', T=1)
title = f"{date} xCut = {xCut}"
fig.suptitle(title)

# Tomography

In [ ]:
hRef   = 3.500
xCut   = 0.000
xMin = -0.1
xMax = 0.1
img_h = {}
for idx in heights:
    hRef = heights[idx]
    print(base_path + f"/{date}_hRef{hRef:.3f}_x{xMin:.3f}_{xMax:.3f}.dat")
    with open(base_path + f"/{date}_hRef{hRef:.3f}_x{xMin:.3f}_{xMax:.3f}.dat", 'rb') as file:
        (imgGeom, img_h[hRef]) = pk.load( file )

In [ ]:
(xMin, xMax, dx, yMin, yMax, dy, zMin, zMax, dz) = imgGeom
nbX = int( (xMax-xMin) / dx ) + 1
nbY = int( (yMax-yMin) / dy ) + 1
nbZ = int( (zMax-zMin) / dz ) + 1
idx0_x = int( nbX / 2 )

## Monostatic

In [ ]:
imgSum  = np.zeros( (nbY, nbZ), dtype=complex)
imgSumC = np.zeros( (nbY, nbZ), dtype=complex)
amp     = np.zeros( (nbY, nbZ), dtype=complex)
idxx = idx0_x
aver = 0

for height in heights:
    for ij in ["21", "31", "32"]:
        z3 = heights[height]
        imgSum +=         img_h[z3][ij][idxx,:,:]
        amp    += np.abs( img_h[z3][ij][idxx,:,:] )
        aver += 1
imgSum = imgSum / aver
print(f"aver = {aver}")

In [ ]:
#imgSumNorm = imgSum / amp
toPlot = 20 * np.log10( np.abs( imgSum ).T )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz)
dia.addColorBar( im, ax, aspect="equal" )

## Bistatic

In [ ]:
biz_imgSum  = np.zeros( (nbY, nbZ), dtype=complex)
biz_imgSumC = np.zeros( (nbY, nbZ), dtype=complex)
biz_amp     = np.zeros( (nbY, nbZ), dtype=complex)
idxx = idx0_x
aver = 0

for idx in heights:
    for ij in ["41", "42", "43"]:
        z3 = heights[idx]
        if z3 != 3.5:
            if ij != "41":
                biz_imgSum +=         img_h[z3][ij][idxx,:,:]
                biz_amp    += np.abs( img_h[z3][ij][idxx,:,:] )
                aver += 1
        else:
            biz_imgSum +=         img_h[z3][ij][idxx,:,:]
            biz_amp    += np.abs( img_h[z3][ij][idxx,:,:] )
            aver += 1
biz_imgSum = biz_imgSum / aver
print(f"aver = {aver}")

In [ ]:
img_h[z3][ij].shape

In [ ]:
#biz_imgSumNorm = biz_imgSum / biz_amp
toPlot = 20 * np.log10( np.abs(biz_imgSum).T )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz)
dia.addColorBar( im, ax, aspect="equal" )

 ## Bistatic sum along x

In [ ]:
biz_amp_i = np.zeros( (nbY, nbZ), dtype=float )

for xn in range(21):
    biz_imgSum  = np.zeros( (nbY, nbZ), dtype=complex )
    # compute the coherent sum
    for idx in heights:
        for ij in ["41", "42", "43"]:
            z3 = heights[idx]
            if z3 != 3.5:
                if ij != "41":
                    biz_imgSum +=         img_h[z3][ij][xn,:,:]
            else:
                biz_imgSum +=         img_h[z3][ij][xn,:,:]
    # compute the non coherent sum
    biz_amp_i = biz_amp_i + biz_imgSum ** 2

In [ ]:
toPlot = 10 * np.log10( np.abs(biz_amp_i).T )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz)
dia.addColorBar( im, ax, aspect="equal" )

## Plot bistatic and monostatic

In [ ]:
fig, grid = dia.plotImgMosaicMonoBi( imgSum.T, biz_imgSum.T, extent_yz, origin='lower' )
title = date
fig.suptitle(title)